In [75]:
#Import required libraries

#!pip install websockets
#!pip install textblob
#!pip install exceptions
#!pip install python-docx
#!pip install aiohttp pillow textblob
#!python -m textblob.download_corpora


In [76]:
#Import Librariers

In [77]:
import asyncio
import aiohttp
from PIL import Image
from io import BytesIO
from textblob import TextBlob

from docx import Document

from docx.shared import Inches  #for image sizing
import os

In [78]:
#Async functions

In [79]:
'''
Opens an async HTTP session to fetch an image from the given URL.
Reads the response as bytes.
Converts the bytes to a PIL.Image object using BytesIO.
Returns the Image object.
'''

'\nOpens an async HTTP session to fetch an image from the given URL.\nReads the response as bytes.\nConverts the bytes to a PIL.Image object using BytesIO.\nReturns the Image object.\n'

In [80]:
async def download_image(image_url):
    """Download image and return PIL Image"""
    async with aiohttp.ClientSession() as session:
        async with session.get(image_url) as resp:
            data = await resp.read()
            image = Image.open(BytesIO(data))
            return image

In [81]:
'''
Opens an async HTTP session.
Fetches the product JSON from DummyJSON.
Returns the list of products.
'''

'\nOpens an async HTTP session.\nFetches the product JSON from DummyJSON.\nReturns the list of products.\n'

In [82]:
async def fetch_products():
    """Fetch products from DummyJSON"""
    api_url = "https://dummyjson.com/products"
    async with aiohttp.ClientSession() as session:
        async with session.get(api_url) as resp:
            data = await resp.json()
            return data['products']

In [83]:
# Synchronous functions

In [84]:
'''
Takes a list of review strings.
If empty → returns 0 sentiment.
Otherwise → calculates average polarity using TextBlob.
polarity ranges from -1 (negative) to +1 (positive).

Returns a single float representing average sentiment.
'''

'\nTakes a list of review strings.\nIf empty → returns 0 sentiment.\nOtherwise → calculates average polarity using TextBlob.\npolarity ranges from -1 (negative) to +1 (positive).\n\nReturns a single float representing average sentiment.\n'

In [85]:
def analyze_reviews(reviews):
    """Simple sentiment analysis using TextBlob"""
    if not reviews:
        return 0.0
    return sum(TextBlob(r).sentiment.polarity for r in reviews) / len(reviews)

In [86]:
'''
Creates a new Word document.
Adds title, price, average sentiment.
Adds each review as a bullet point.

If an image is provided:
Saes it temporarily as temp_image.png.
Inserts into the document.

Deletes the temporary file.
Saves the DOCX using the product title as filename.

Returns the filename.
'''

'\nCreates a new Word document.\nAdds title, price, average sentiment.\nAdds each review as a bullet point.\n\nIf an image is provided:\nSaes it temporarily as temp_image.png.\nInserts into the document.\n\nDeletes the temporary file.\nSaves the DOCX using the product title as filename.\n\nReturns the filename.\n'

In [87]:
def generate_docx(product, sentiment, image=None):
    """Generate DOCX report for a single product"""
    doc = Document()
    doc.add_heading(product["title"], 0)
    doc.add_paragraph(f"Price: ${product['price']}")
    doc.add_paragraph(f"Average sentiment: {sentiment:.2f}")
    doc.add_paragraph("Reviews:")
    for r in product.get("reviews", []):
        doc.add_paragraph(r, style='List Bullet')
    
    if image:
        # Save image temporarily
        img_path = "temp_image.png"
        image.save(img_path)
        doc.add_picture(img_path, width=Inches(2))
        os.remove(img_path)
    
    filename = f"{product['title'].replace(' ', '_')}.docx"
    doc.save(filename)
    return filename


In [88]:
#Test function

In [89]:
'''
test_analyze_reviews :checks sentiment is within valid range (-1 to 1).
test_generate_docx :creates a dummy DOCX with a dummy image, checks file exists, then deletes it.
'''

'\ntest_analyze_reviews :checks sentiment is within valid range (-1 to 1).\ntest_generate_docx :creates a dummy DOCX with a dummy image, checks file exists, then deletes it.\n'

In [90]:
def test_analyze_reviews():
    reviews = ["Good", "Bad", "Excellent"]
    sentiment = analyze_reviews(reviews)
    assert -1 <= sentiment <= 1
    print("test_analyze_reviews passed")

In [91]:
def test_generate_docx():
    from PIL import Image
    product = {
        "title": "Test Product",
        "price": 9.99,
        "reviews": ["Nice"]
    }
    sentiment = 0.5
    image = Image.new('RGB', (150, 150), color='blue')  # Dummy image
    filename = generate_docx(product, sentiment, image)
    assert os.path.exists(filename)
    os.remove(filename)
    print("test_generate_docx passed")

In [92]:
async def test_download_image():
    url = "https://via.placeholder.com/150"
    try:
        image = await download_image(url)
        assert isinstance(image, Image.Image)
        print("test_download_image passed")
    except Exception as e:
        print("test_download_image skipped (cannot download online):", e)

In [93]:
async def test_fetch_products():
    try:
        products = await fetch_products()
        assert isinstance(products, list)
        print("test_fetch_products passed")
    except Exception as e:
        print("test_fetch_products skipped (cannot fetch online):", e)

In [94]:
#Run test

In [96]:
'''
Manual test calls in Jupyter notebook.
Skips tests that need online access if network is blocked.
'''

'\nManual test calls in Jupyter notebook.\nSkips tests that need online access if network is blocked.\n'

In [95]:
test_analyze_reviews()
test_generate_docx()
await test_download_image()
await test_fetch_products()
print("All tests completed")

test_analyze_reviews passed
test_generate_docx passed
test_download_image skipped (cannot download online): Cannot connect to host via.placeholder.com:443 ssl:default [getaddrinfo failed]
test_fetch_products passed
All tests completed
